In [56]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from itertools import product
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from googletrans import Translator
translator = Translator()
import re

# from xgboost import XGBRegressor
# from xgboost import plot_importance

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

import time
import sys
import gc
import pickle
sys.version_info

sys.version_info(major=3, minor=6, micro=10, releaselevel='final', serial=0)

In [57]:
items = pd.read_csv('items.csv')
shops = pd.read_csv('shops.csv')
cats = pd.read_csv('item_categories.csv')
train = pd.read_csv('sales_train.csv')
# set index to ID to avoid droping it later
test  = pd.read_csv('test.csv').set_index('ID')

In [58]:
# Need to translate the russian name into english for each shop in shops file
def translate_shop_name(row):
#     shp_id = int(row['shop_id'])
    shop_name_rs = row['shop_name']
    shop_name_en = translator.translate(shop_name_rs).text
    shop_name_en = re.sub('[^a-zA-Z0-9 \n\.\ ]', "", shop_name_en).strip()
#     shop_name_en = re.sub("[!@#$%^&*()[]{};:,./<>?\|`~-=_+]", "", shop_name_en)
#     print(shop_name_en)
    return shop_name_en

In [59]:
# trans = translator.translate(shops['shop_name'].iloc[0])
# trans.text.replace("\'","")
# shops_translate = []
# for i in range(len(shops)):
#     shops_translate.append((i, translator.translate(shops['shop_name'].iloc[i]).text))
# shops_translate
shops['shop_name_en'] = shops.apply(translate_shop_name, axis = 1)
shops['city'] = shops['shop_name_en'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == ' Yakutsk', 'city'] = 'Yakutsk'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
# shops = shops[['shop_id','city_code']]
# shops


,shop_name,shop_id,shop_name_en,city,city_code
0,"!Якутск Орджоникидзе, 56 фран",0,Yakutsk Ordzhonikidze 56 Franc,Yakutsk,28
1,"!Якутск ТЦ ""Центральный"" фран",1,Yakutsk TC Central Franc,Yakutsk,28
2,"Адыгея ТЦ ""Мега""",2,Adygea TC Mega,Adygea,0
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Balashikha TRC OctoberKinomir,Balashikha,1
4,"Волжский ТЦ ""Волга Молл""",4,Volzhsky mall Volga Mall,Volzhsky,26
5,"Вологда ТРЦ ""Мармелад""",5,Vologda SEC Marmalade,Vologda,25
6,"Воронеж (Плехановская, 13)",6,Voronezh Plekhanovskaya 13,Voronezh,27
7,"Воронеж ТРЦ ""Максимир""",7,Voronezh TRC Maksimir,Voronezh,27
8,"Воронеж ТРЦ Сити-Парк ""Град""",8,Voronezh TRC CityPark City,Voronezh,27
9,Выездная Торговля,9,Itinerant trade,Itinerant,4


In [60]:
shops = shops[['shop_id','city_code']]

In [61]:
# Need to translate the russian name into english for each shop in shops file
def translate_item_category_name(row):
#     shp_id = int(row['shop_id'])
    cat_name_rs = row['item_category_name']
    cat_name_en = translator.translate(cat_name_rs).text
    cat_name_en = re.sub('[^a-zA-Z0-9 \n\.\ ]', "", cat_name_en)
    cat_name_en = re.sub(' +', ' ', cat_name_en)
#     shop_name_en = re.sub("[!@#$%^&*()[]{};:,./<>?\|`~-=_+]", "", shop_name_en)
#     print(shop_name_en)
    return cat_name_en

In [62]:
cats['item_category_name_en'] = cats.apply(translate_item_category_name, axis = 1)

In [63]:
cats['type'] = cats['item_category_name_en'].str.split(' ').map(lambda x: x[0])
cats['type_code'] = LabelEncoder().fit_transform(cats['type'])
cats['subtype'] = cats['item_category_name_en'].str.split(' ').map(lambda x: x[1] if len(x) > 1 else x[0])
cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])

In [9]:
cats = cats[['item_category_id','type_code', 'subtype_code']]

In [10]:
train = train[train.item_price<100000]
train = train[train.item_cnt_day<1001]

In [11]:
train.loc[train.item_price<0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
484683,15.05.2013,4,32,2973,-1.0,1.0


In [12]:
median = train[(train.shop_id==32)&(train.item_id==2973)&(train.date_block_num==4)&(train.item_price>0)].item_price.median()
train.loc[train.item_price<0, 'item_price'] = median
train.loc[train.item_price<0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day


In [13]:
len(list(set(test.item_id) - set(test.item_id).intersection(set(train.item_id)))), len(list(set(test.item_id))), len(test)

(363, 5100, 214200)

In [14]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [18]:
train['date'] = train['date'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y'))

train['day'] = train.date.apply(lambda x: int(x.strftime('%d')))
train['month'] = train.date.apply(lambda x: int(x.strftime('%m')))
train['year'] = train.date.apply(lambda x: int(x.strftime('%Y')))

In [55]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,revenue
2052769,2014-10-01,21,2,1485,1799.0,1.0,1,10,2014,1799.0
2052807,2014-10-01,21,2,1905,249.0,1.0,1,10,2014,249.0
2052707,2014-10-01,21,2,3340,1499.0,2.0,1,10,2014,2998.0
2052907,2014-10-01,21,2,3343,2599.0,2.0,1,10,2014,5198.0
2052934,2014-10-01,21,2,3732,2799.0,1.0,1,10,2014,2799.0
...,...,...,...,...,...,...,...,...,...,...
2098955,2014-10-31,21,59,20034,549.0,1.0,31,10,2014,549.0
2099049,2014-10-31,21,59,20242,299.0,1.0,31,10,2014,299.0
2099052,2014-10-31,21,59,20654,1399.0,1.0,31,10,2014,1399.0
2099031,2014-10-31,21,59,22087,79.0,1.0,31,10,2014,79.0


In [21]:
train.to_csv('training_v1.csv', index=False)

In [54]:
train['revenue'] = train['item_price'] *  train['item_cnt_day']
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,revenue
2052769,2014-10-01,21,2,1485,1799.0,1.0,1,10,2014,1799.0
2052807,2014-10-01,21,2,1905,249.0,1.0,1,10,2014,249.0
2052707,2014-10-01,21,2,3340,1499.0,2.0,1,10,2014,2998.0
2052907,2014-10-01,21,2,3343,2599.0,2.0,1,10,2014,5198.0
2052934,2014-10-01,21,2,3732,2799.0,1.0,1,10,2014,2799.0
...,...,...,...,...,...,...,...,...,...,...
2098955,2014-10-31,21,59,20034,549.0,1.0,31,10,2014,549.0
2099049,2014-10-31,21,59,20242,299.0,1.0,31,10,2014,299.0
2099052,2014-10-31,21,59,20654,1399.0,1.0,31,10,2014,1399.0
2099031,2014-10-31,21,59,22087,79.0,1.0,31,10,2014,79.0
